# MobileNetV3 fine-tuning

## Imports

In [1]:
from pathlib import Path

import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from sklearn.model_selection import GroupShuffleSplit

from src.data.loaders import KDEFDataModule
from src.models.trainer import EmotionClassifier

torch.set_float32_matmul_precision("high")

## Data Preparation

#### We discard images of angles `FullRight` and `FullLeft` and bugged files.

In [2]:
BUGGED_FILES = [
    "AF01SUFR",
    "AF10AFFR",
    "AF11NEHL",
    "AF20DIHL",
    "AM25DIFL",
    "AM34DIFR",
    "BF13NEHR",
    "BM21DIFL",
    "BM22DIHL",
    "BM24DIFL",
]
target_angles = ["S", "HL", "HR"]
all_files = []

for p in Path("data/raw").rglob("*.JPG"):
    name = p.stem
    angle = name[6:]

    if name in BUGGED_FILES:
        print("Skipped buggy file:", p)
        continue

    if angle in target_angles:
        all_files.append(p)

Skipped buggy file: data\raw\AF01\AF01SUFR.JPG
Skipped buggy file: data\raw\AF10\AF10AFFR.JPG
Skipped buggy file: data\raw\AF11\AF11NEHL.JPG
Skipped buggy file: data\raw\AF20\AF20DIHL.JPG
Skipped buggy file: data\raw\AM25\AM25DIFL.JPG
Skipped buggy file: data\raw\AM34\AM34DIFR.JPG
Skipped buggy file: data\raw\BF13\BF13NEHR.JPG
Skipped buggy file: data\raw\BM21\BM21DIFL.JPG
Skipped buggy file: data\raw\BM22\BM22DIHL.JPG
Skipped buggy file: data\raw\BM24\BM24DIFL.JPG


#### We perform a subject-wise split to ensure that images from the same subject do not appear in different sets.

In [3]:
df = pd.DataFrame({"path": all_files})

df["subject_id"] = df["path"].apply(lambda x: x.name[1:4])
df["session"] = df["path"].apply(lambda x: x.name[0])

print(f"Number of images: {len(df)}")
print(f"Number of unique subjects: {df['subject_id'].nunique()}")

Number of images: 2936
Number of unique subjects: 70


In [4]:
splitter = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=0)
train_idx, temp_idx = next(splitter.split(df, groups=df["subject_id"]))

train_df = df.iloc[train_idx]
temp_df = df.iloc[temp_idx]

splitter_val = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
val_idx, test_idx = next(splitter_val.split(temp_df, groups=temp_df["subject_id"]))

val_df = temp_df.iloc[val_idx]
test_df = temp_df.iloc[test_idx]

In [5]:
print("-" * 30)
print(
    f"Train set: {len(train_df)} images (Subjects: {train_df['subject_id'].nunique()})"
)
print(f"Val set:   {len(val_df)} images (Subjects: {val_df['subject_id'].nunique()})")
print(f"Test set:  {len(test_df)} images (Subjects: {test_df['subject_id'].nunique()})")

train_ids = set(train_df["subject_id"].unique())
val_ids = set(val_df["subject_id"].unique())
test_ids = set(test_df["subject_id"].unique())

print("-" * 30)
print(f"Intersection Train/Val: {train_ids.intersection(val_ids)}")
print(f"Intersection Train/Test: {train_ids.intersection(test_ids)}")

------------------------------
Train set: 2181 images (Subjects: 52)
Val set:   377 images (Subjects: 9)
Test set:  378 images (Subjects: 9)
------------------------------
Intersection Train/Val: set()
Intersection Train/Test: set()


### Model Training

In [6]:
data_module = KDEFDataModule(
    train_df,
    val_df,
    test_df,
    batch_size=32,
)

checkpoint_callback = ModelCheckpoint(
    dirpath="outputs/models",
    filename="mobilenet_v3_kdef-frozen-{epoch:02d}-{val_f1:.2f}",
    save_top_k=1,
    monitor="val_f1",
    mode="max",
)

model_frozen = EmotionClassifier(num_classes=7, learning_rate=1e-3, freeze_backbone=True)

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    logger=CSVLogger("outputs", name="logs"),
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


INFO: Backbone frozen. Training classifier head only.


In [7]:
trainer.fit(model_frozen, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ MobileNetV3               │  4.2 M │ train │     0 │
│ 1 │ train_acc     │ MulticlassAccuracy        │      0 │ train │     0 │
│ 2 │ val_acc       │ MulticlassAccuracy        │      0 │ train │     0 │
│ 3 │ val_f1        │ MulticlassF1Score         │      0 │ train │     0 │
│ 4 │ val_precision │ MulticlassPrecision       │      0 │ train │     0 │
│ 5 │ val_recall    │ MulticlassRecall          │      0 │ train │     0 │
│ 6 │ conf_mat      │ MulticlassConfusionMatrix │      0 │ train │     0 │
└───┴───────────────┴───────────────────────────┴────────┴───────┴───────┘

Trainable params: 1.2 M                                                                                            
Non-trainable params: 3.0 M                                                                                        
Total params: 4.2 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 261                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=20` reached.


In [8]:
model_unfrozen = EmotionClassifier(num_classes=7, learning_rate=1e-3, freeze_backbone=False)

checkpoint_callback = ModelCheckpoint(
    dirpath="outputs/models",
    filename="mobilenet_v3_kdef-unfrozen-{epoch:02d}-{val_f1:.2f}",
    save_top_k=1,
    monitor="val_f1",
    mode="max",
)

trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    devices=1,
    logger=CSVLogger("outputs", name="logs"),
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


In [9]:
trainer.fit(model_unfrozen, datamodule=data_module)

d:\studia\AffectiveAI\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:881: Checkpoint directory D:\studia\AffectiveAI\outputs\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ MobileNetV3               │  4.2 M │ train │     0 │
│ 1 │ train_acc     │ MulticlassAccuracy        │      0 │ train │     0 │
│ 2 │ val_acc       │ MulticlassAccuracy        │      0 │ train │     0 │
│ 3 │ val_f1        │ MulticlassF1Score         │      0 │ train │     0 │
│ 4 │ val_precision │ MulticlassPrecision       │      0 │ train │     0 │
│ 5 │ val_recall    │ MulticlassRecall          │      0 │ train │     0 │
│ 6 │ conf_mat      │ MulticlassConfusionMatrix │      0 │ train │     0 │
└───┴───────────────┴───────────────────────────┴────────┴───────┴───────┘

Trainable params: 4.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.2 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 261                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.
